# this script is used to convert cityscape dataset to:
# .png and .txt files that can be used with yolov5

In [10]:
import os
import json

annotation_root  = "cityscape_dataset/gtFine_trainvaltest/gtFine"
images_root = "cityscape_dataset/leftImg8bit_trainvaltest/leftImg8bit"

images_final_destination = "datasets/cityscape/images"
labels_final_destination = "datasets/cityscape/labels"

In [15]:
# move files

for split in os.listdir(images_root):
    split_path = os.path.join(images_root, split)
    for city in os.listdir(split_path):
        files_path = os.path.join(split_path, city)
        for file in os.listdir(files_path):
            image_path = os.path.join(files_path, file)
            name = file.split("_left")[0] + ".png"
            new_name = os.path.join(images_final_destination, name)
            os.rename(image_path, new_name)


In [16]:
"""
from cityscape website
Group	Classes
----------------------------------
flat	road · sidewalk · parking+ · rail track+
human	person* · rider*
vehicle	car* · truck* · bus* · on rails* · motorcycle* · bicycle* · caravan*+ · trailer*+
construction	building · wall · fence · guard rail+ · bridge+ · tunnel+
object	pole · pole group+ · traffic sign · traffic light
nature	vegetation · terrain
sky	sky
void	ground+ · dynamic+ · static+
"""

name_to_number = {
    "road": 0,
    "sidewalk": 0,

    "person": 1,
    "persongroup": 1,
    "ridergroup": 1,
    "rider": 1,

    "car": 2,
    "truck": 2,
    "truckgroup": 2,
    "bus": 2,
    "motorcycle": 2,
    "motorcyclegroup": 2,
    "bicycle": 2,
    "bicyclegroup": 2,
    "caravan": 2,
    "trailer": 2,
    "cargroup": 2,

    "building": 3,
    "wall": 3,
    "fence": 3,
    "guard rail": 3,
    "bridge": 3,
    "tunnel": 3,

    "pole": 4,
    "polegroup": 4,
    "traffic sign": 4,
    "traffic light": 4,

    "terrain": 5,
    "vegetation": 5,

    "sky": 6,

    "static": 7,
    "dynamic": 7,
    "ground": 7,

    # unknown:
    "license plate": -1,
    'ego vehicle': -1,
    'rectification border': -1,
    'out of roi': -1,
    "parking": -1,
    "train": -1,
    "rail track": -1,
}

In [23]:
# create labels for images

for split in os.listdir(annotation_root):
    split_path = os.path.join(annotation_root, split)
    for city in os.listdir(split_path):
        files_path = os.path.join(split_path, city)
        for file in os.listdir(files_path):
            if file.endswith(".json"):
                file_path = os.path.join(files_path, file)
                name = file.split("_gtFine")[0] + ".txt"
                rows = []
                with open(file_path, "r") as f:
                    json_obj = json.load(f)
                    h = json_obj['imgHeight']
                    w = json_obj['imgWidth']
                    obs = json_obj['objects']
                    for o in obs:
                        label = o["label"]
                        label_number = name_to_number[label]
                        if label_number != -1:
                            coords = o["polygon"]
                            max_w = max(coords, key=lambda x : x[0])[0]
                            min_w = min(coords, key=lambda x : x[0])[0]
                            max_h = max(coords, key=lambda x : x[1])[1]
                            min_h = min(coords, key=lambda x : x[1])[1]
                            w_center = ((min_w + max_w) / 2) / w
                            w_size = (max_w - min_w) / w
                            h_center = ((min_h + max_h) / 2) / h
                            h_size = (max_h - min_h) / h
                            line = f"{label_number} {w_center:.6f} {h_center:.6f} {w_size:.6f} {h_size:.6f}\n"
                            rows.append(line)
                with open(os.path.join(labels_final_destination, name), "w") as f:
                    f.writelines(rows)